# batch 优化算法      

## mini-batch梯度下降     
应用于巨大的数据集。假设一个数据集$X,Y=[x^{(1)},x^{(2)}...x^{(m-1)},x^{(m)}],[y^{(1)},y^{(2)}...y^{(m-1)},y^{(m)}],m=5\times10^6$     

将该数据集划分为$5000$个mini-batch，每个batch包含$1000$个样本，即$X^{\{t\}}=[x^{(1000t-999)},x^{(1000t-998)}...x^{(1000t-1)},x^{(1000t)}],Y^{\{t\}}=[y^{(1000t-999)},y^{(1000t-998)}...y^{(1000t-1)},y^{(1000t)}]$       

其中$X^{\{t\}}$代表第$t$个batch,整个数据集包含$\frac{m}{t}$个batch，遍历所有batch(一次遍历了数据集)是一个epoch。**mini-batch梯度下降就是在每个batch上进行梯度下降。**

- **mini-batch梯度下降的理解**    
mini-batch梯度下降与梯度下降不同，mini-batch的成本函数总体趋势下降，但会存在噪声。噪声来自于训练过程中的某些batch可能时易于分类的，因此相比之下成本函数较低。
<img style="float: center;" src="course_2_pics/week_2_1.png" width=500 height=500>      

1. 如果$batch=m$，即梯度下降法；对于较大的训练集而言速度很慢，精度高。
2. 如果$batch=1$，即随机梯度下降法；速度快(但无法向量化，降低了速度)，但随机梯度下降可能不会收敛到最小值，也可能收敛后在最小值附近波动。   
3. 如果$1<batch<m$，即mini-batch梯度下降法，保证了精度和速度(除了batch-size较小以外，还可以向量化)。

- **batch选择原则**   
1. 训练集小于2000个样本：使用batch梯度下降法。
2. 样本数目较大：一般的mini-batch大小为64到512，考虑到电脑内存设置和使用的方式，如果mini-batch大小是2的次方，代码运行较快。    

# 梯度下降算法

## 指数加权平均 (EWMA)   

- **指数移动加权平均法**:         
加权移动平均法，是对观察值分别给予不同的权数，按不同权数求得移动平均值，并以最后的移动平均值为基础，确定预测值的方法。采用加权移动平均法，是因为观察期的近期观察值对预测值有较大影响，它更能反映近期变化的趋势。**指数移动加权平均法**，是指各数值的加权系数随时间呈**指数式递减**。(在计算中占用更小的储存量)


一个时间序列$\theta_t=\{\theta_1,\theta_2....\theta_T\}$,令$V_0=0,V_t=\beta V_{t-1}+(1-\beta)\theta_t$，得到的序列$V_t$即指数平滑序列：        

$$
\begin{aligned}
V_t&=\beta V_{t-1}+(1-\beta)\theta_t\\
&=\beta V_{t-2}+\beta(1-\beta) \theta_{t-1}+(1-\beta)\theta_t\\
&=\beta V_0+\beta^{t-1}(1-\beta)\theta_{1}+\beta^{t-2}(1-\beta)\theta_{2}+...\beta(1-\beta) \theta_{t-1}+(1-\beta)\theta_t
\end{aligned}
$$

绿色的曲线代表$\beta$值较大，红色的$\beta$值中等，黄色的$\beta$值非常小    
- 绿色曲线历史数据的权重较大，相对而言稳定，较为平滑，预测不够准确。           
- 黄色曲线的历史数据权重较小，更注重当前数据，噪声较大。
<img style="float: center;" src="course_2_pics/week_2_2.png" width=300 height=300>     

- **指数加权移动平均的偏差修正**      
当令$V_0=0$时，会发现预测初始阶段刚的预测值较小，如图中紫色线(而不是绿色线)     
要修正其偏差，只需要在预测初始阶段不使用$V_t$，而使用$\frac{V_t}{1-\beta ^t}$进行预测(计算$V_{t+1}$仍然使用$V_t$)
<img style="float: center;" src="course_2_pics/week_2_3.png" width=500 height=500>          

## 动量梯度下降法(Mometum)      
如果你要优化成本函数，函数形状如图，红点代表最小值的位置,假设进行梯度下降法。     
- 如蓝线所所表示的梯度下降过程，在梯度下降过程中不能总是沿着全局最小的方向下降——在纵轴方向有较大曲折，而理想情况是沿着横轴。并且在优化末期，不能够使用较大的学习率，会导致跳出局部最优。


- 红线是mometum算法的下降过程，mometum算法通过计算$V_{dw}=\beta V_{dw}+(1-\beta dw)$，$db$同理，更新参数时$w=w-\alpha V_{dw}$。(不容易受初始偏差的影响)
<img style="float: center;" src="course_2_pics/week_2_4.png" width=700 height=700>      

## RMSprop(root mean square prop)     
假设一个简单的情况，在下图中纵轴是$b$，横轴是$w$，那么希望减少纵轴的波动时，算法如下：

$
On\ iteration\ t\\
通过mini-batch计算db,dw\\
S_{dw}=\beta_2S_{dw}+(1-\beta_2)(dw)^2,S_{db}=\beta_2S_{db}+(1-\beta_2)(db)^2\\
w=w-\alpha\cfrac{dw}{\sqrt{S_{dw}+\varepsilon}},b=b-\alpha\cfrac{db}{\sqrt{S_{db}+\varepsilon}}
$
由于$db$较大，因此$S_{db}$较大，在RMS下，最终$b$的移动距离较小；$w$相反。
<img style="float: center;" src="course_2_pics/week_2_5.png" width=500 height=500>

## Adam
- **算法:**    
初始条件:$V_{dw}=0,S_{dw}=0,S_{db}=0,S_{db}=0$      
在第$t$次迭代时:     
1. 根据mini-batch计算$dw,db$     
2. $V_{dw}=\beta_1 V_{dw}+(1-\beta_1) dw,V_{db}=\beta_1 V_{db}+(1-\beta_1) db$
3. $S_{dw}=\beta_2S_{dw}+(1-\beta_2)(dw)^2,S_{db}=\beta_2S_{db}+(1-\beta_2)(db)^2$     
4. 计算偏差修正:$V_{dw}^{corrected}=\cfrac{V_{dw}}{1-(\beta_1)^t},V_{db}^{corrected}=\cfrac{V_{db}}{1-(\beta_1)^t},S_{dw}^{corrected}=\cfrac{S_{dw}}{1-(\beta_2)^t},S_{db}^{corrected}=\cfrac{S_{db}}{1-(\beta_2)^t}$     
5. 更新参数：$w=w-\alpha\cfrac{V_{dw}^{corrected}}{\sqrt{S_{dw}^{corrected}+\varepsilon}},b=b-\alpha\cfrac{V_{db}^{corrected}}{\sqrt{S_{db}^{corrected}+\varepsilon}}$    

## 学习率衰减     
几种方式:   
1. $\alpha=\cfrac{\alpha_0}{decay\ rate\times epoch}$

2. $0.95^{epoch}\alpha_0$    

3.$\cfrac{k\alpha_0}{\sqrt{epoch}}$        

## 局部最优    
在高维空间中，不太可能陷入局部最优而容易陷入鞍点   


# 模型调试      
## 超参调试      
超参数(按重要性)：
1. 学习率$\alpha$
2. 梯度下降算法参数$\beta_1,\beta_2,\varepsilon$，神经元数$\# units$，小批量梯度下降的批量大小$mini-batch\ size$
3. 神经网络层数$\# layers$与,训练集遍历次数$epoch$，学习率衰减速率$decay\ rate$       


早一代的机器学习算法中，如果你有两个超参数：超参1，超参2，常见的做法是**在网格中取样点**。深度学习领域，常做的是**随机选择点**。因为对于问题而言，很难提前知道哪个超参数最重要。

- 例子：          
假设超参数1是学习速率$\alpha$,假设超参数2是Adam算法中$\varepsilon$。$\alpha$的取值很重要，而$\varepsilon$取值则无关紧要。如果在网格中取点，你试验了$\alpha$的5个取值，无论$\varepsilon$取何值，对于$\alpha$结果基本上都是一样的。共有25种模型，但进行试验的值只有5个。相对的，如果你随机取值，你会试验25个独立的$\alpha$，更有可能发现效果做好的那个。

<img style="float: center;" src="course_2_pics/week_3_1.png" width=500 height=500>     

进一步的调参可以通过从**粗到细**的挑选，首先随机选择，之后缩小范围调参。      

## 超参数调整范围      
两个例子，对于较为重要的超参数$\alpha$与$\beta$:      
1. 假设搜索超参数$\alpha$（学习速率），取值范围[0.0001,1]。画一条从0.0001到1的数轴，沿其随机均匀取值，那90%的数值将会落在0.1到1之间，结果就是，**在0.1到1之间，应用了90%的资源，而在0.0001到0.1之间，只有10%的搜索资源**。因此，用**对数标尺**搜索超参数的方式会更合理，因此这里不使用线性轴，分别依次取0.0001，0.001，0.01，0.1，1，在对数轴上均匀随机取。
```
r=-4*np.random.rand()
a=10^r
```
2. 另一个例子是给$\beta$取值，用于计算指数的加权平均值。假设你认为是0.9到0.999之间的某个值。如果在0.9到0.999区间搜索，那就不能用线性轴取值。因此探索$(1-\beta)$的取值，此值在0.1到0.001区间内。      

之所以利用对数轴，是因为在整个数轴上，当超参数取不同值时，算法对其灵敏度可能不同(由于指数等原因)     




## Softmax回归         
分类器需要分类多个类别时使用Softmax回归,神经网络最后接一有$C$(类别数)个节点的层，每个节点输出预测值是每个类的概率,概率相加等于1。       
<img style="float: center;" src="course_2_pics/week_3_6.png" width=500 height=500>       
对于softmax层计算$z^{[l]}=w^{[l]}a^{[l-1]}+b^{[l]}$，通过激活函数计算中间变量$t=e^{z^{[l]}}$，最后$a^{[l]}=\cfrac{e^{z^{[l]}}}{\sum_i t_i},a_i^{[l]}=\cfrac{t_i}{\sum_i t_i}$输出一个$\hat y$           

- **softmax的训练**      
假设$y=[1,0,0,0]^T,\hat y=[0.8,0.1,0.05,0.05]$，损失函数$L=\sum_i^4y_ilog(\hat y_i)$，对于目标值而言，类别中只有一个值为1，其余都为0，那么损失函数可以转化为$L=y_jlog(\hat y_j),y_j\neq0$，成本函数为$J=\frac{1}{m}\sum_k^mL(\hat y^{[k]},y^{[k]})$